In [25]:
import pandas as pd
import numpy as np
import requests
import json

from bs4 import BeautifulSoup as bf
from typing import Tuple, List, Dict


In [26]:
api_key = '{My_Key}'
path=r"C:\Users\David Sofer\Desktop\David\python"
fileName="\destinations_LP_crawler_Ex5.csv"
try:
    df=pd.read_csv(path+fileName,encoding='ISO-8859-1')
except:
    print("File not found, try again")
df

,city,city_LP
0,Antalya,mediterranean-coast/antalya
1,Athens,Athens
2,Ayia-napa,the-republic-of-cyprus/Agia-napa
3,Bangkok,Bangkok
4,Barcelona,Barcelona
...,...,...
71,Venice,Venice
72,Vienna,Vienna
73,Warsaw,Warsaw
74,Zakynthos,Zakynthos


In [27]:
def get_city_data(address: list, api_key: str):
    url = "https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address,api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error", response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("requests.get doesn't work properly")

    return response_data

In [6]:
countries_list=[]
for row in df.itertuples(index=True, name='df'):
    get_country = get_city_data(row.city, api_key)
    if get_country['status']=='ZERO_RESULTS':
        countries_list.append("NULL")
        continue
    countries_list.append(get_country['results'][0]['formatted_address'].split(',')[-1].strip().replace(" ",""))
countries_list

['Turkey',
 'Greece',
 'Cyprus',
 'Thailand',
 'Spain',
 'Switzerland',
 'Georgia',
 'China',
 'Germany',
 'USA',
 'Slovakia',
 'Belgium',
 'Romania',
 'Hungary',
 'Bulgaria',
 'Mexico',
 'Italy',
 'Thailand',
 'Germany',
 'Greece',
 'Spain',
 'Greece',
 'NULL',
 'India',
 'Germany',
 'Ireland',
 'Croatia',
 'Germany',
 'Italy',
 'Germany',
 'Switzerland',
 'China',
 'Germany',
 'Austria',
 'Turkey',
 'Greece',
 'Thailand',
 '02000',
 'Cyprus',
 'USA',
 'Cyprus',
 'Portugal',
 'Slovenia',
 'USA',
 'Spain',
 'Philippines',
 'France',
 'Mexico',
 'USA',
 'Italy',
 'Russia',
 'India',
 'Germany',
 'Greece',
 'Italy',
 'France',
 '65000',
 'France',
 'Thailand',
 'Greece',
 'Italy',
 'Austria',
 'USA',
 'China',
 'Bulgaria',
 'Russia',
 'Germany',
 'Taiwan',
 'Georgia',
 'Greece',
 'Canada',
 'Italy',
 'Austria',
 'Poland',
 'Greece',
 'Switzerland']

In [29]:
df.insert(2, 'Countries', countries_list)

In [30]:
df

,city,city_LP,Countries
0,Antalya,mediterranean-coast/antalya,Turkey
1,Athens,Athens,Greece
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,Cyprus
3,Bangkok,Bangkok,Thailand
4,Barcelona,Barcelona,Spain
...,...,...,...
71,Venice,Venice,Italy
72,Vienna,Vienna,Austria
73,Warsaw,Warsaw,Poland
74,Zakynthos,Zakynthos,Greece


In [31]:
from bs4 import BeautifulSoup as bf

def get_country_description(city_url):
    try:
        response = requests.get(city_url)
        if not response.status_code == 200:
            print("HTTP error", response.status_code)
        else:
            try:
                soup = bf(response.content,'html.parser')
                for div in soup.find_all("div", class_ = "readMore_content___5EuR relative overflow-hidden max-h-96 is-max wysiwyg"):
                    return div.get_text()
            except:
                print("Response not in valid HTML format")
    except:
        print("requests.get doesn't work properly")



In [32]:
df.insert(3, 'Description', "NULL")

In [33]:
for index, row in df.iterrows():
    city_url = "https://www.lonelyplanet.com"
    city_url +="/" +row["Countries"]+ "/" + row["city_LP"]
    row["Description"]=get_country_description(city_url)

HTTP error 403
HTTP error 403
HTTP error 403


In [34]:
df

,city,city_LP,Countries,Description
0,Antalya,mediterranean-coast/antalya,Turkey,Once seen simply as the gateway to the Turkish...
1,Athens,Athens,Greece,"With equal measures of grunge and grace, Athen..."
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,Cyprus,Endless blue-sky days and a coastline riddled ...
3,Bangkok,Bangkok,Thailand,"Same same, but different. This Thailish T-shir..."
4,Barcelona,Barcelona,Spain,Barcelona is an enchanting seaside city with b...
...,...,...,...,...
71,Venice,Venice,Italy,Imagine the audacity of building a city of mar...
72,Vienna,Vienna,Austria,Baroque streetscapes and imperial palaces set ...
73,Warsaw,Warsaw,Poland,"A phoenix arisen from the ashes, Poland's capi..."
74,Zakynthos,Zakynthos,Greece,"Zakynthos, also known by its Italian name Zant..."


In [35]:
data = {'City': df['city'], 'Country': df['Countries'], 'Description': df['Description']}
detailed_frame = pd.DataFrame(data)
detailed_frame.to_csv(r"C:\Users\David Sofer\Desktop\David\python\LP_destinations.csv")


In [36]:
detailed_frame

,City,Country,Description
0,Antalya,Turkey,Once seen simply as the gateway to the Turkish...
1,Athens,Greece,"With equal measures of grunge and grace, Athen..."
2,Ayia-napa,Cyprus,Endless blue-sky days and a coastline riddled ...
3,Bangkok,Thailand,"Same same, but different. This Thailish T-shir..."
4,Barcelona,Spain,Barcelona is an enchanting seaside city with b...
...,...,...,...
71,Venice,Italy,Imagine the audacity of building a city of mar...
72,Vienna,Austria,Baroque streetscapes and imperial palaces set ...
73,Warsaw,Poland,"A phoenix arisen from the ashes, Poland's capi..."
74,Zakynthos,Greece,"Zakynthos, also known by its Italian name Zant..."
